## EOPatch

### Create an empty patch 

In [26]:
from eolearn.core import EOPatch, FeatureType
import numpy as np

# Set a feature to EOPatch. Each feature has to belong to one of the feature types listed above.
patch = EOPatch()
new_bands = np.zeros((5, 10, 10, 13), dtype=np.float32)

In [27]:
patch[FeatureType.DATA]['bands'] = new_bands
# or patch.data['bands'] = new_bands

In [28]:
# Check current content of EOPatch with it’s string representation.
patch

EOPatch(
  data: {
    bands: numpy.ndarray(shape=(5, 10, 10, 13), dtype=float32)
  }
  mask: {}
  scalar: {}
  label: {}
  vector: {}
  data_timeless: {}
  mask_timeless: {}
  scalar_timeless: {}
  label_timeless: {}
  vector_timeless: {}
  meta_info: {}
  bbox: None
  timestamp: []
)

In [29]:
# Get all non-empty features of EOPatch
patch.get_features()

# Get a feature from EOPatch
data = patch[FeatureType.DATA]['bands']
# or patch.data['bands']

In [30]:
# Save EOPatch to local folder. In case EOPatch would already exist in the specified location we are also giving a permission to overwrite its features.
from eolearn.core import OverwritePermission

patch.save('./example_patch', overwrite_permission=OverwritePermission.OVERWRITE_FEATURES)

In [31]:
# Compare EOPatches
patch == patch2

False

In [32]:
# Remove a feature from EOPatch
del patch2[FeatureType.DATA]['bands']
# or del patch.data['bands']

### Make a shallow and deep copy of EOPatch. Shallow copy will copy only a reference to data but not the data itself.

In [33]:
patch1 = patch.__copy__()
patch2 = patch.__deepcopy__()

patch.data['bands'] += 1

patch == patch1, patch == patch2

(True, False)

### Load EOPatch from the same folder

In [34]:
patch2 = EOPatch.load('./example_patch')

### Concatenate two EOPatches

In [35]:
patch2[FeatureType.DATA]['bands2'] = new_bands

patch + patch2

EOPatch(
  data: {
    bands: numpy.ndarray(shape=(10, 10, 10, 13), dtype=float32)
    bands2: numpy.ndarray(shape=(5, 10, 10, 13), dtype=float32)
  }
  mask: {}
  scalar: {}
  label: {}
  vector: {}
  data_timeless: {}
  mask_timeless: {}
  scalar_timeless: {}
  label_timeless: {}
  vector_timeless: {}
  meta_info: {}
  bbox: None
  timestamp: []
)

## EOTask

### Add a feature using the EOTask

In [36]:
from eolearn.core import EOPatch, FeatureType, AddFeature # AddFeature is a simple EOTask which adds a feature to a given EOPatch
import numpy as np

patch = EOPatch()

feature = (FeatureType.DATA, 'bands')
add_feature = AddFeature(feature)

data = np.zeros((5, 100, 100, 13))
patch = add_feature.execute(patch, data)
# or patch = add_feature(patch, data)

patch


EOPatch(
  data: {
    bands: numpy.ndarray(shape=(5, 100, 100, 13), dtype=float64)
  }
  mask: {}
  scalar: {}
  label: {}
  vector: {}
  data_timeless: {}
  mask_timeless: {}
  scalar_timeless: {}
  label_timeless: {}
  vector_timeless: {}
  meta_info: {}
  bbox: None
  timestamp: []
)

### Create a composite task using a multiplication operator (a * b) function

In [37]:
from eolearn.core import CopyTask, RenameFeature

copy_task = CopyTask()
rename_feature = RenameFeature((FeatureType.DATA, 'bands', 'the_bands'))
copy_rename_task = rename_feature * copy_task

new_patch = copy_rename_task(patch)
new_patch

EOPatch(
  data: {
    the_bands: numpy.ndarray(shape=(5, 100, 100, 13), dtype=float64)
  }
  mask: {}
  scalar: {}
  label: {}
  vector: {}
  data_timeless: {}
  mask_timeless: {}
  scalar_timeless: {}
  label_timeless: {}
  vector_timeless: {}
  meta_info: {}
  bbox: None
  timestamp: []
)

### EOTask Class

In [38]:
from eolearn.core import EOTask

class Example_Task(EOTask):
    def __init__(self, example_param):
        self.example_param = example_param
    
    def execute(self, eopatch, *, patch_specific_param=None):
        return eopatch


## EOWorkflow

### Create Workflow

In [39]:
from eolearn.core import EOWorkflow

workflow = EOWorkflow([
    (add_feature, []),
    (copy_task, [add_feature]),
    (rename_feature, [copy_task])
])

result = workflow.execute({
    add_feature: {'eopatch': patch, 'data': new_bands}
})